In [15]:
import os

interface_dir = os.environ["DATA"] + "webinterfaces/exp02/"

tasks_dir = os.path.join(interface_dir, "res", "tasks")
results_dir = os.path.join(interface_dir, "results")
protocols_dir = os.path.join(interface_dir, "protocols")
prolific_matching_dir = os.path.join(interface_dir, "prolific")
demographics_files = [
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_696f78d1e5c51c6b1f1d67b4.csv"),
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_6970a4330ba6485c07050d6d.csv"),
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_6970dcc3c9493748d49aae7b.csv"),
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_6971ddd924ae95f00c488534.csv"),
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_6978bd06e8976381a9546691.csv"),
    os.path.join(interface_dir, "prolific", "demographics", "prolific_demographic_export_6979fb556f8962cd22b7f6d3.csv"),
]

protocol_paths_d = {
    "H": os.path.join(protocols_dir, "H_0.json"),
    "H+AI": os.path.join(protocols_dir, "AI_0.json"),
    "H+AI+CF": os.path.join(protocols_dir, "XAI_CF_0.json"),
    "H+AI+SHAP": os.path.join(protocols_dir, "XAI_SHAP_0.json"),
    "H+AI+LLM": os.path.join(protocols_dir, "XAI_LLM_0.json"),
    "H+AI+GRADCAM": os.path.join(protocols_dir, "XAI_GRADCAM_0.json"),
}

COMPREHENSION_THRESHOLD = 0.8

COMPREHENSION_TASKS = ["xeasy1_find_pattern_rot"]
TRAINING_TASKS = ["med3_find_pattern_rot"]
EASY_TASKS = ["easy1_find_pattern_rot", "easy3_find_pattern_rot"]
DIFFICULT_TASKS = ["hard1_find_pattern_rot", "hard3_find_pattern_rot"]

MILD_PRESSURE_TASKS = ["easy1_find_pattern_rot", "hard1_find_pattern_rot"]
STRONG_PRESSURE_TASKS = ["easy3_find_pattern_rot", "hard3_find_pattern_rot"]

TASK_PROTOCOL_KEYS = {
    "easy1_find_pattern_rot": "mainexp_easy_mild_patrot_task",
    "easy3_find_pattern_rot": "mainexp_easy_strong_patrot_task",

    "hard1_find_pattern_rot": "mainexp_hard_mild_patrot_task",
    "hard3_find_pattern_rot": "mainexp_hard_strong_patrot_task",

    "xeasy1_find_pattern_rot": "intro_comprehension_task",
    "med3_find_pattern_rot": "intro_training_1_task"
}


In [16]:
import sys

sys.path.append("/home/jleguy/Documents/postdoc/git_repos/WebXAII/")

In [17]:
import pandas as pd
from typing import List

def concatenate_csvs_by_participant(csv_paths):
    """
    Reads multiple CSV files and concatenates them into a single DataFrame,
    using "Participant id" as the key.

    Args:
        csv_paths (List[str]): List of paths to CSV files.

    Returns:
        pd.DataFrame: Concatenated DataFrame with "Participant id" as the key.
    """
    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(path) for path in csv_paths]

    # Concatenate all DataFrames
    concatenated_df = pd.concat(dfs, ignore_index=True)

    # Set "Participant id" as the index (dictionary key)
    concatenated_df.set_index("Participant id", inplace=True)

    return concatenated_df


In [18]:
demographics_df = concatenate_csvs_by_participant(demographics_files)

In [19]:
import json
import csv
import numpy as np


def load_json(path):
    with open(path) as json_file:
        return json.load(json_file)


def load_task_csv_file(path):
    y_true, y_pred = [], []
    with open(path) as csv_data:
        reader = csv.DictReader(csv_data)
        for row in reader:
            y_true.append(int(row["target"]))
            y_pred.append(int(row["pred"]))

    return np.array(y_true), np.array(y_pred)


In [20]:
import json


def data_matching(protocols_paths_d, prolific_matching_files):
    results_filenames_d = {k: [] for k in protocol_paths_d.keys()}

    for prolific_matching_file in prolific_matching_files:

        with open(prolific_matching_file) as json_data:
            d = json.load(json_data)

            for prolific_id, prot_dict in d.items():
                condition_split = prot_dict["protocol"].split("_")
                filename = prolific_id + ".json"

                if condition_split[0] == "H":
                    results_filenames_d["H"].append(filename)
                elif condition_split[0] == "AI":
                    results_filenames_d["H+AI"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "SHAP":
                    results_filenames_d["H+AI+SHAP"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "CF":
                    results_filenames_d["H+AI+CF"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "LLM":
                    results_filenames_d["H+AI+LLM"].append(filename)
                elif condition_split[0] == "XAI" and condition_split[1] == "GRADCAM":
                    results_filenames_d["H+AI+GRADCAM"].append(filename)

    return results_filenames_d


In [21]:
from pywebxaii.resretrieval import extract_p_questionnaire_results, get_protocol_entry_from_key


def extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys, expected_answers_length=None):
    output_res_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_d[group_key] = {"raw": {}, "values": {}, "times": {}}

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            data_issue = False
            if not curr_res_d["is_completed"]:
                data_issue = True

            # Iterating over all questionnaires keys
            for quest_key in quest_keys:

                curr_protocol_d = load_json(protocol_paths_d[group_key])
                try:
                    get_protocol_entry_from_key(curr_protocol_d, quest_key)
                    answers_raw, answers_values, quest_times = extract_p_questionnaire_results(curr_res_d,
                                                                                               quest_key,
                                                                                               protocol_d=curr_protocol_d)
                    if expected_answers_length is not None and len(answers_raw) != expected_answers_length:
                        print(f"error for file {filename}")

                except KeyError:
                    data_issue = True

                if quest_key not in output_res_d[group_key]["raw"]:
                    output_res_d[group_key]["raw"][quest_key] = []
                    output_res_d[group_key]["values"][quest_key] = []
                    output_res_d[group_key]["times"][quest_key] = []

                if data_issue:
                    output_res_d[group_key]["raw"][quest_key].append(None)
                    output_res_d[group_key]["values"][quest_key].append(None)
                    output_res_d[group_key]["times"][quest_key].append(None)
                else:
                    output_res_d[group_key]["raw"][quest_key].append(answers_raw)
                    output_res_d[group_key]["values"][quest_key].append(answers_values)
                    output_res_d[group_key]["times"][quest_key].append(quest_times)

    return output_res_d

In [22]:
from pywebxaii.resretrieval import extract_p_task_results


def compute_scores(results_dir, results_filenames_d, protocol_paths_d, tasks_dir, tasks_names, task_protocol_keys):
    output_res_scores_d = {}
    output_res_reliance_d = {}
    output_res_overreliance_d = {}
    output_res_underreliance_d = {}
    output_res_appropriate_reliance_d = {}
    output_res_task_true_d = {}
    output_res_ai_pred_d = {}
    output_res_user_decision_d = {}
    output_res_quest_order_d = {}
    output_res_answer_times_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        output_res_scores_d[group_key] = []
        output_res_reliance_d[group_key] = []
        output_res_overreliance_d[group_key] = []
        output_res_underreliance_d[group_key] = []
        output_res_appropriate_reliance_d[group_key] = []
        output_res_answer_times_d[group_key] = []
        output_res_task_true_d[group_key] = []
        output_res_ai_pred_d[group_key] = []
        output_res_user_decision_d[group_key] = []
        output_res_quest_order_d[group_key] = []

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            if not curr_res_d["is_completed"]:
                continue

            nb_questions = 0
            nb_quest_wrong_predictions = 0
            nb_quest_right_predictions = 0
            nb_correct = 0
            nb_reliance = 0
            nb_overreliance = 0
            nb_underreliance = 0
            answer_times = []
            early_break = False
            task_true_l = []
            ai_pred_l = []
            user_decision_l = []
            quest_order_l = []
            # Iterating over all tasks
            for task_idx, task_name in enumerate(tasks_names):

                task_true, ai_pred = load_task_csv_file(os.path.join(tasks_dir, task_name + "_content.csv"))

                answers_idx_vect, answers_text_vect, quest_order_vect, time_vect, _, _ = \
                    extract_p_task_results(curr_res_d,
                                           task_protocol_keys[tasks_names[task_idx]],
                                           protocol_d=load_json(protocol_paths_d[group_key]))

                nb_questions += len(answers_idx_vect)
                nb_quest_wrong_predictions += np.sum(task_true != ai_pred)
                nb_quest_right_predictions += np.sum(task_true == ai_pred)

                try:
                    nb_correct += np.sum(answers_idx_vect == np.logical_not(task_true))
                    nb_reliance += np.sum(answers_idx_vect == np.logical_not(ai_pred))
                    nb_overreliance += np.sum(np.logical_and(
                        answers_idx_vect == np.logical_not(ai_pred),
                        ai_pred != task_true
                    ))
                    nb_underreliance += np.sum(np.logical_and(
                        answers_idx_vect != np.logical_not(ai_pred),
                        ai_pred == task_true
                    ))
                    answer_times.extend((np.array(time_vect) / 1000).tolist())

                    task_true_l.extend(np.array(task_true, dtype=bool).tolist())
                    ai_pred_l.extend(np.array(ai_pred, dtype=bool).tolist())
                    user_decision_l.extend(np.logical_not(answers_idx_vect).tolist())
                    quest_order_l.extend(quest_order_vect.tolist())

                    #
                    # if np.isnan(np.sum(answers_idx_vect)):
                    #     print(f"answers {answers_idx_vect}")
                    #     print(f"true {np.logical_not(task_true)}")
                    #     print(f"ai pred {np.logical_not(ai_pred)}")
                    #     print(f"correct extracted {np.sum(answers_idx_vect == np.logical_not(task_true))}")
                    #     print(f"reliance extracted {np.sum(answers_idx_vect == np.logical_not(ai_pred))}")

                # Happens if the results file is not complete
                except ValueError:
                    print("ValueError exception")
                    output_res_scores_d[group_key].append(None)
                    output_res_reliance_d[group_key].append(None)
                    output_res_overreliance_d[group_key].append(None)
                    output_res_underreliance_d[group_key].append(None)
                    output_res_answer_times_d[group_key].append(None)
                    output_res_task_true_d[group_key].append(None)
                    output_res_ai_pred_d[group_key].append(None)
                    output_res_user_decision_d[group_key].append(None)
                    output_res_quest_order_d[group_key].append(None)
                    early_break = True
                    break

            if not early_break:
                output_res_scores_d[group_key].append(nb_correct / nb_questions)
                output_res_reliance_d[group_key].append(nb_reliance / nb_questions)
                output_res_overreliance_d[group_key].append(nb_overreliance / nb_quest_wrong_predictions)
                output_res_underreliance_d[group_key].append(nb_underreliance / nb_quest_right_predictions)
                output_res_answer_times_d[group_key].append(answer_times)
                output_res_task_true_d[group_key].append(task_true_l)
                output_res_ai_pred_d[group_key].append(ai_pred_l)
                output_res_user_decision_d[group_key].append(user_decision_l)
                output_res_quest_order_d[group_key].append(quest_order_l)

    return (output_res_scores_d, output_res_reliance_d, output_res_overreliance_d, output_res_underreliance_d, output_res_answer_times_d,
            output_res_task_true_d, output_res_ai_pred_d, output_res_user_decision_d, output_res_quest_order_d)


In [23]:
results_filenames_d = data_matching(protocol_paths_d, [os.path.join(prolific_matching_dir, "prolific.json"),
                                                       os.path.join(prolific_matching_dir, "prolific_21-1.json"),
                                                       os.path.join(prolific_matching_dir, "prolific_21-2.json"),
                                                       os.path.join(prolific_matching_dir, "prolific_22-1.json"),
                                                       os.path.join(prolific_matching_dir, "prolific_27-1.json"),
                                                       os.path.join(prolific_matching_dir, "prolific_28-1.json"),
                                                       ])

# results_filenames_d = data_matching(protocol_paths_d, [os.path.join(prolific_matching_dir, "prolific_28-1.json")])

In [24]:
total = 0
for k, v in results_filenames_d.items():
    total += len(v)
    print(f"{k}: {len(v)}")
print(f"Total: {total}")

H: 121
H+AI: 106
H+AI+CF: 115
H+AI+SHAP: 113
H+AI+LLM: 110
H+AI+GRADCAM: 108
Total: 673


In [25]:
def filter_not_completed(results_filenames_d):
    filtered_results_filenames_d = {}

    # Iterating over all groups
    for group_key, filenames_list in results_filenames_d.items():

        filtered_results_filenames_d[group_key] = []

        # Iterating on all results files for the current group
        for filename in filenames_list:
            curr_res_path = os.path.join(results_dir, filename)
            curr_res_d = load_json(curr_res_path)
            if not curr_res_d["is_completed"]:
                continue
            filtered_results_filenames_d[group_key].append(filename)
    return filtered_results_filenames_d

In [26]:
results_filenames_d = filter_not_completed(results_filenames_d)

In [27]:
results_filenames_before_filtering = dict(results_filenames_d)

In [28]:
total = 0
for k, v in results_filenames_d.items():
    total += len(v)
    print(f"{k}: {len(v)}")
print(f"Total: {total}")

H: 99
H+AI: 97
H+AI+CF: 100
H+AI+SHAP: 97
H+AI+LLM: 98
H+AI+GRADCAM: 93
Total: 584


In [29]:
def filter_on_attention_tests(results_filenames_d):
    res = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d,
                                ["attentioncheck_1", "attentioncheck_2"], expected_answers_length=2)
    filtered_results_filenames_d = {}

    for k, v in res.items():
        filtered_results_filenames_d[k] = []
        for i in range(len(results_filenames_d[k])):
            if v["raw"]["attentioncheck_1"][i] is None:
                passes1 = False
            else:
                passes1 = v["raw"]["attentioncheck_1"][i][0] == 2 and v["raw"]["attentioncheck_1"][i][1] == 0

            if v["raw"]["attentioncheck_2"][i] is None:
                passes2 = False
            else:
                passes2 = v["raw"]["attentioncheck_2"][i][0] == 6 and v["raw"]["attentioncheck_2"][i][1] == 0

            if passes1 and passes2:
                filtered_results_filenames_d[k].append(results_filenames_d[k][i])

    return filtered_results_filenames_d

In [30]:
results_filenames_d = filter_on_attention_tests(results_filenames_d)

In [31]:
total = 0
for k, v in results_filenames_d.items():
    total += len(v)
    print(f"{k}: {len(v)}")
print(f"Total: {total}")

H: 88
H+AI: 92
H+AI+CF: 89
H+AI+SHAP: 91
H+AI+LLM: 94
H+AI+GRADCAM: 88
Total: 542


In [32]:
print("Success rate at attention tests :")
for k, v in results_filenames_d.items():
    total += len(v)
    print(f"{k}: {len(v)/len(results_filenames_before_filtering[k])*100:.2f}%")

Success rate at attention tests :
H: 88.89%
H+AI: 94.85%
H+AI+CF: 89.00%
H+AI+SHAP: 93.81%
H+AI+LLM: 95.92%
H+AI+GRADCAM: 94.62%


In [33]:
def filter_comprehension_score(results_filenames_d):
    comprehension_score_d, _, _, _, _, _, _, _, _= compute_scores(results_dir, results_filenames_d, protocol_paths_d, tasks_dir,
                                                       COMPREHENSION_TASKS, TASK_PROTOCOL_KEYS)
    filtered_results_filenames_d = {}

    for k, v in comprehension_score_d.items():
        filtered_results_filenames_d[k] = []
        for i in range(len(results_filenames_d[k])):
            if comprehension_score_d[k][i] >= COMPREHENSION_THRESHOLD:
                filtered_results_filenames_d[k].append(results_filenames_d[k][i])
            else:
                print(f"Rejecting sample due to comprehension score of {comprehension_score_d[k][i]}")

    return filtered_results_filenames_d


In [34]:
results_filenames_d = filter_comprehension_score(results_filenames_d)

Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.4
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to comprehension score of 0.6
Rejecting sample due to compreh

In [35]:
total = 0
for k, v in results_filenames_d.items():
    total += len(v)
    print(f"{k}: {len(v)}")
print(f"Total: {total}")

H: 85
H+AI: 87
H+AI+CF: 80
H+AI+SHAP: 86
H+AI+LLM: 84
H+AI+GRADCAM: 83
Total: 505


In [36]:
print(
    f"Total passing filters among complete files: {np.sum([len(v) for v in results_filenames_d.values()])}/{np.sum([len(v) for v in results_filenames_before_filtering.values()])}")

Total passing filters among complete files: 505/584


In [37]:
505/600

0.8416666666666667

In [38]:
import json

def extract_view_order_list_from_file(json_path, target_view_ids):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    index_by_view = {}

    if "data" in data:
        views_list = data["data"]
    else:
        views_list = data

    for idx, d in enumerate(views_list):
        if isinstance(d, dict) and "view_id" in d:
            index_by_view[d["view_id"]] = idx

    return [index_by_view.get(v) for v in target_view_ids]


def get_task_order(data):
    """
    data: dict[str, list[list]]

    Returns:
        dict with same structure, but values replaced by within-list ranks
    """
    result = {}

    views_names = ["easy_mild", "easy_strong", "hard_mild", "hard_strong"]

    for key, lists in data.items():
        tasks_order = []

        for lst in lists:
            task_order = [None for v in views_names]

            sorted_vals = sorted(lst)
            rank = {v: i + 1 for i, v in enumerate(sorted_vals)}
            ranked_list = [rank[v]-1 for v in lst]
            for i, rank in enumerate(ranked_list):
                task_order[ranked_list[i]] = views_names[i]

            tasks_order.append(task_order)

        result[key] = tasks_order

    return result

def extract_tasks_view_ids(results_filenames):
    views_ids = ["mainexp_easy_mild_patrot_task", "mainexp_easy_strong_patrot_task", "mainexp_hard_mild_patrot_task",
                 "mainexp_hard_strong_patrot_task"]

    data = {}
    for k, v in results_filenames.items():
        data[k] = []
        for filename in v:
            extracted_data = extract_view_order_list_from_file(os.path.join(results_dir, filename), views_ids)
            data[k].append(extracted_data)

    return data


In [39]:
def compute_cogload_vectors(cogload_results):

    cogload_all_values_easy_mild = {}
    cogload_all_values_easy_strong = {}
    cogload_all_values_hard_mild = {}
    cogload_all_values_hard_strong = {}
    cogload_easy_mild = {}
    cogload_easy_strong = {}
    cogload_hard_mild = {}
    cogload_hard_strong = {}

    for group_key, v in cogload_results.items():
        cogload_all_values_easy_mild[group_key] = []
        cogload_all_values_easy_strong[group_key] = []
        cogload_all_values_hard_mild[group_key] = []
        cogload_all_values_hard_strong[group_key] = []
        cogload_easy_mild[group_key] = []
        cogload_easy_strong[group_key] = []
        cogload_hard_mild[group_key] = []
        cogload_hard_strong[group_key] = []

        for session_id, results_list in v["raw"].items():
            for result_arr in results_list:
                if session_id == "mainexp_easy_mild_cogload":
                    cogload_all_values_easy_mild[group_key].append(result_arr)
                    cogload_easy_mild[group_key].append(np.sum(result_arr))
                elif session_id == "mainexp_easy_strong_cogload":
                    cogload_all_values_easy_strong[group_key].append(result_arr)
                    cogload_easy_strong[group_key].append(np.sum(result_arr))
                elif session_id == "mainexp_hard_mild_cogload":
                    cogload_all_values_hard_mild[group_key].append(result_arr)
                    cogload_hard_mild[group_key].append(np.sum(result_arr))
                elif session_id == "mainexp_hard_strong_cogload":
                    cogload_all_values_hard_strong[group_key].append(result_arr)
                    cogload_hard_strong[group_key].append(np.sum(result_arr))

    return cogload_easy_mild, cogload_easy_strong, cogload_hard_mild, cogload_hard_strong, cogload_all_values_easy_mild, cogload_all_values_easy_strong, cogload_all_values_hard_mild, cogload_all_values_hard_strong


In [40]:
quest_keys = ["mainexp_easy_mild_cogload", "mainexp_easy_strong_cogload", "mainexp_hard_mild_cogload",
              "mainexp_hard_strong_cogload"]

cogload_results = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys, expected_answers_length=6)

In [41]:
cogload_easy_mild, cogload_easy_strong, cogload_hard_mild, cogload_hard_strong, cogload_all_values_easy_mild, cogload_all_values_easy_strong, cogload_all_values_hard_mild, cogload_all_values_hard_strong = compute_cogload_vectors(cogload_results)

In [42]:
cogload_all_values_easy_strong["H+AI"]

[array([6, 5, 5, 4, 6, 5], dtype=object),
 array([5, 1, 6, 2, 5, 6], dtype=object),
 array([4, 1, 6, 2, 4, 5], dtype=object),
 array([6, 3, 6, 4, 6, 4], dtype=object),
 array([5, 0, 5, 4, 5, 5], dtype=object),
 array([5, 5, 5, 3, 5, 3], dtype=object),
 array([5, 5, 6, 3, 5, 6], dtype=object),
 array([5, 0, 5, 2, 1, 3], dtype=object),
 array([6, 2, 6, 3, 6, 4], dtype=object),
 array([4, 4, 4, 3, 5, 5], dtype=object),
 array([0, 0, 0, 3, 0, 0], dtype=object),
 array([6, 2, 6, 2, 5, 6], dtype=object),
 array([6, 1, 6, 3, 6, 3], dtype=object),
 array([5, 1, 5, 2, 5, 2], dtype=object),
 array([2, 0, 5, 1, 6, 1], dtype=object),
 array([6, 0, 6, 4, 6, 3], dtype=object),
 array([6, 1, 6, 3, 6, 3], dtype=object),
 array([2, 5, 5, 1, 6, 2], dtype=object),
 array([5, 1, 6, 3, 6, 5], dtype=object),
 array([3, 1, 6, 1, 4, 6], dtype=object),
 array([6, 3, 6, 1, 6, 6], dtype=object),
 array([6, 0, 6, 3, 6, 5], dtype=object),
 array([6, 0, 6, 2, 6, 2], dtype=object),
 array([5, 0, 5, 1, 2, 0], dtype=o

In [43]:
results_filenames_d["H+AI"][45]

'61572aeab6ac8a5fe94b43b4.json'

In [71]:
def extract_trust_data(results_dir, results_filenames_d, protocol_paths_d):

    trust_easy_mild = {}
    trust_easy_strong = {}
    trust_hard_mild = {}
    trust_hard_strong = {}

    quest_keys = ["mainexp_easy_mild_reliance", "mainexp_easy_strong_reliance", "mainexp_hard_mild_reliance",
                  "mainexp_hard_strong_reliance"]

    data_input = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, quest_keys, expected_answers_length=1)

    for key_group, data in data_input.items():
        trust_easy_mild[key_group] = []
        trust_easy_strong[key_group] = []
        trust_hard_mild[key_group] = []
        trust_hard_strong[key_group] = []

        for form_key in quest_keys:

            if form_key not in data["raw"]:
                continue

            for arr in data["raw"][form_key]:
                trust = arr[0] if arr is not None else None

                if form_key == "mainexp_easy_mild_reliance":
                    trust_easy_mild[key_group].append(trust)
                elif form_key == "mainexp_easy_strong_reliance":
                    trust_easy_strong[key_group].append(trust)
                elif form_key == "mainexp_hard_mild_reliance":
                    trust_hard_mild[key_group].append(trust)
                elif form_key == "mainexp_hard_strong_reliance":
                    trust_hard_strong[key_group].append(trust)

    return trust_easy_mild, trust_easy_strong, trust_hard_mild, trust_hard_strong


In [72]:
trust_easy_mild, trust_easy_strong, trust_hard_mild, trust_hard_strong = extract_trust_data(results_dir, results_filenames_d, protocol_paths_d)


In [73]:
def extract_closing_quest_strategy_data(results_dir, results_filenames_d, protocol_paths_d):
    output_dict = {}
    d = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, ["closing_quest_strategy"], expected_answers_length=1)

    for group_key, data in d.items():
        output_dict[group_key] = []
        for v in data["values"]["closing_quest_strategy"]:
            output_dict[group_key].append(v[0] if v is not None else None)
    return output_dict


In [74]:
quest_strategy = extract_closing_quest_strategy_data(results_dir, results_filenames_d, protocol_paths_d)

In [80]:
def extract_xai_helpfulness(results_dir, results_filenames_d, protocol_paths_d):
    verify_answers_d = {}
    understand_ai_processes_d = {}
    ai_errors_detection = {}

    d = extract_quest_results(results_dir, results_filenames_d, protocol_paths_d, ["closing_quest_use_xai"], expected_answers_length=3)

    for group_key, data in d.items():
        verify_answers_d[group_key] = []
        understand_ai_processes_d[group_key] = []
        ai_errors_detection[group_key] = []

        for v in data["raw"]["closing_quest_use_xai"]:
            verify_answers_d[group_key].append(v[0] if v is not None else None)
            understand_ai_processes_d[group_key].append(v[1] if v is not None else None)
            ai_errors_detection[group_key].append(v[2] if v is not None else None)

    return verify_answers_d, understand_ai_processes_d, ai_errors_detection


In [81]:
verify_answer_helpfulness, understand_ai_processes_helpfulness, ai_errors_detection_helpfulness =  extract_xai_helpfulness(results_dir, results_filenames_d, protocol_paths_d)

In [82]:
all_scores, all_reliance, all_overreliance, all_underreliance, all_times, \
    all_task_true, all_ai_pred, all_user_decision, all_quest_order = compute_scores(results_dir,
                                                                                          results_filenames_d,
                                                                                          protocol_paths_d,
                                                                                          tasks_dir,
                                                                                          EASY_TASKS + DIFFICULT_TASKS,
                                                                                          TASK_PROTOCOL_KEYS)

In [83]:
scores_easy_mild, reliance_easy_mild, overreliance_easy_mild, underreliance_easy_mild, answertimes_easy_mild, \
    task_true_easy_mild, ai_pred_easy_mild, user_decision_easy_mild, quest_order_easy_mild = compute_scores(
    results_dir, results_filenames_d, protocol_paths_d, tasks_dir, ["easy1_find_pattern_rot"], TASK_PROTOCOL_KEYS)
scores_easy_strong, reliance_easy_strong, overreliance_easy_strong, underreliance_easy_strong, answertimes_easy_strong, \
    task_true_easy_strong, ai_pred_easy_strong, user_decision_easy_strong, quest_order_easy_strong = compute_scores(
    results_dir, results_filenames_d, protocol_paths_d, tasks_dir, ["easy3_find_pattern_rot"], TASK_PROTOCOL_KEYS)
scores_hard_mild, reliance_hard_mild, overreliance_hard_mild, underreliance_hard_mild, answertimes_hard_mild, \
    task_true_hard_mild, ai_pred_hard_mild, user_decision_hard_mild, quest_order_hard_mild = compute_scores(
    results_dir, results_filenames_d, protocol_paths_d, tasks_dir, ["hard1_find_pattern_rot"], TASK_PROTOCOL_KEYS)
scores_hard_strong, reliance_hard_strong, overreliance_hard_strong, underreliance_hard_strong, answertimes_hard_strong, \
    task_true_hard_strong, ai_pred_hard_strong, user_decision_hard_strong, quest_order_hard_strong = compute_scores(
    results_dir, results_filenames_d, protocol_paths_d, tasks_dir, ["hard3_find_pattern_rot"], TASK_PROTOCOL_KEYS)

comprehension_score, _, _, _, _, _, _, _, _ = compute_scores(results_dir, results_filenames_d, protocol_paths_d, tasks_dir,
                                                 COMPREHENSION_TASKS, TASK_PROTOCOL_KEYS)


In [84]:
tasks_order = get_task_order(extract_tasks_view_ids(results_filenames_d))

In [85]:
import pandas as pd

rows = []

for key in all_scores.keys():
    # For each key, iterate over all participants
    for i in range(len(all_scores[key])):

        participant_id = results_filenames_d[key][i].split(".")[0]

        row = {
            "participant_id": participant_id,
            "sex": demographics_df.loc[participant_id, "Sex"],
            "age": demographics_df.loc[participant_id, "Age"],
            "xai_condition": key,
            "tasks_order": tasks_order[key][i],
            "comprehension_score": comprehension_score[key][i],

            "score_full": all_scores[key][i],
            "reliance_full": all_reliance[key][i],
            "overreliance_full": all_overreliance[key][i],
            "underreliance_full": all_underreliance[key][i],
            "answer_times_full": all_times[key][i],
            "task_true_full": all_task_true[key][i],
            "ai_pred_full": all_ai_pred[key][i],
            "user_decision_full": all_user_decision[key][i],
            "quest_order_full": all_quest_order[key][i],

            "score_easy_mild": scores_easy_mild[key][i],
            "reliance_easy_mild": reliance_easy_mild[key][i],
            "overreliance_easy_mild": overreliance_easy_mild[key][i],
            "underreliance_easy_mild": underreliance_easy_mild[key][i],
            "answer_times_easy_mild": answertimes_easy_mild[key][i],
            "task_true_easy_mild": task_true_easy_mild[key][i],
            "ai_pred_easy_mild": ai_pred_easy_mild[key][i],
            "user_decision_easy_mild": user_decision_easy_mild[key][i],
            "quest_order_easy_mild": quest_order_easy_mild[key][i],

            "score_easy_strong": scores_easy_strong[key][i],
            "reliance_easy_strong": reliance_easy_strong[key][i],
            "overreliance_easy_strong": overreliance_easy_strong[key][i],
            "underreliance_easy_strong": underreliance_easy_strong[key][i],
            "answer_times_easy_strong": answertimes_easy_strong[key][i],
            "task_true_easy_strong": task_true_easy_strong[key][i],
            "ai_pred_easy_strong": ai_pred_easy_strong[key][i],
            "user_decision_easy_strong": user_decision_easy_strong[key][i],
            "quest_order_easy_strong": quest_order_easy_strong[key][i],

            "score_hard_mild": scores_hard_mild[key][i],
            "reliance_hard_mild": reliance_hard_mild[key][i],
            "overreliance_hard_mild": overreliance_hard_mild[key][i],
            "underreliance_hard_mild": underreliance_hard_mild[key][i],
            "answer_times_hard_mild": answertimes_hard_mild[key][i],
            "task_true_hard_mild": task_true_hard_mild[key][i],
            "ai_pred_hard_mild": ai_pred_hard_mild[key][i],
            "user_decision_hard_mild": user_decision_hard_mild[key][i],
            "quest_order_hard_mild": quest_order_hard_mild[key][i],

            "score_hard_strong": scores_hard_strong[key][i],
            "reliance_hard_strong": reliance_hard_strong[key][i],
            "overreliance_hard_strong": overreliance_hard_strong[key][i],
            "underreliance_hard_strong": underreliance_hard_strong[key][i],
            "answer_times_hard_strong": answertimes_hard_strong[key][i],
            "task_true_hard_strong": task_true_hard_strong[key][i],
            "ai_pred_hard_strong": ai_pred_hard_strong[key][i],
            "user_decision_hard_strong": user_decision_hard_strong[key][i],
            "quest_order_hard_strong": quest_order_hard_strong[key][i],

            "cogload_easy_mild": cogload_easy_mild[key][i],
            "cogload_easy_strong": cogload_easy_strong[key][i],
            "cogload_hard_mild": cogload_hard_mild[key][i],
            "cogload_hard_strong": cogload_hard_strong[key][i],
            "cogload_all_values_easy_mild": cogload_all_values_easy_mild[key][i],
            "cogload_all_values_easy_strong": cogload_all_values_easy_strong[key][i],
            "cogload_all_values_hard_mild": cogload_all_values_hard_mild[key][i],
            "cogload_all_values_hard_strong": cogload_all_values_hard_strong[key][i],

            "trust_easy_mild": trust_easy_mild[key][i],
            "trust_easy_strong": trust_easy_strong[key][i],
            "trust_hard_mild": trust_hard_mild[key][i],
            "trust_hard_strong": trust_hard_strong[key][i],

            "quest_verify_answer_helpfulness": verify_answer_helpfulness[key][i],
            "quest_understand_ai_processes_helpfulness": understand_ai_processes_helpfulness[key][i],
            "quest_ai_errors_detection_helpfulness": ai_errors_detection_helpfulness[key][i],

            "quest_ai_strategy" : quest_strategy[key][i],
        }
        rows.append(row)

# Create the DataFrame
df = pd.DataFrame(rows)

df.set_index("participant_id", inplace=True)

In [86]:
def save_df(df, output_dir):

    # Save to CSV
    df.to_csv(os.path.join(output_dir, "filtered_data.csv"), index=True)
    # Save to Pickle
    df.to_pickle(os.path.join(output_dir, "filtered_data.pkl"))

In [87]:
save_df(df, interface_dir)

In [88]:
df

,sex,age,xai_condition,tasks_order,comprehension_score,score_full,reliance_full,overreliance_full,underreliance_full,answer_times_full,...,cogload_all_values_hard_mild,cogload_all_values_hard_strong,trust_easy_mild,trust_easy_strong,trust_hard_mild,trust_hard_strong,quest_verify_answer_helpfulness,quest_understand_ai_processes_helpfulness,quest_ai_errors_detection_helpfulness,quest_ai_strategy
participant_id,,,,,,,,,,,,,,,,,,,,,
5a33e4a10758280001030ce5,Female,35,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",1.0,0.729167,0.645833,0.250,0.275,"[3.7, 6.2, 9.6, 5.8, 7.2, 12.7, 4.6, 4.2, 12.1...",...,"[4, 1, 4, 3, 5, 4]","[6, 2, 6, 5, 5, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5bcb95ffb592580001ec24cf,Female,43,H,"[hard_mild, easy_mild, easy_strong, hard_strong]",1.0,0.770833,0.666667,0.125,0.225,"[4.5, 8.2, 9.6, 7.7, 5.2, 14.5, 10.1, 6.4, 4.0...",...,"[6, 4, 6, 3, 6, 5]","[6, 6, 6, 5, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
631745a70c7c585248df1a5d,Male,33,H,"[hard_mild, hard_strong, easy_mild, easy_strong]",0.8,0.708333,0.625000,0.125,0.275,"[7.0, 10.0, 16.6, 20.6, 13.5, 10.9, 9.5, 10.5,...",...,"[3, 0, 6, 3, 5, 4]","[5, 0, 6, 3, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5fd8ea198512a801831f6b85,Male,40,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",1.0,0.770833,0.687500,0.250,0.225,"[6.7, 5.7, 7.4, 4.4, 5.6, 10.0, 7.6, 4.6, 9.8,...",...,"[5, 4, 3, 4, 4, 4]","[6, 6, 6, 6, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
66d035e529e80315f590ecce,Female,46,H,"[hard_mild, easy_strong, hard_strong, easy_mild]",1.0,0.666667,0.625000,0.375,0.325,"[6.8, 7.8, 8.8, 7.0, 9.3, 9.1, 6.1, 6.0, 9.7, ...",...,"[5, 1, 4, 3, 5, 4]","[6, 1, 6, 4, 5, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ed9ee3afd7ffe59917d3dfa,Female,34,H+AI+GRADCAM,"[hard_strong, hard_mild, easy_strong, easy_mild]",1.0,0.916667,0.750000,0.000,0.100,"[3.0, 4.0, 7.6, 5.4, 6.0, 4.6, 2.6, 4.7, 2.5, ...",...,"[4, 0, 2, 1, 4, 1]","[5, 0, 6, 2, 5, 2]",2.0,2.0,4.0,1.0,6.0,6.0,4.0,I sometimes or often relied on the AI’s predic...
63d7f1983e82a895912b3b32,Male,42,H+AI+GRADCAM,"[hard_strong, easy_mild, hard_mild, easy_strong]",1.0,0.916667,0.812500,0.125,0.050,"[3.4, 5.0, 4.0, 6.2, 3.1, 4.3, 6.8, 4.4, 10.9,...",...,"[4, 1, 5, 3, 5, 5]","[3, 1, 6, 3, 4, 5]",4.0,3.0,3.0,4.0,5.0,1.0,1.0,I sometimes or often relied on the AI’s predic...
587b38ec3962c30001276c81,Female,64,H+AI+GRADCAM,"[hard_strong, easy_strong, easy_mild, hard_mild]",0.8,0.916667,0.875000,0.375,0.025,"[7.2, 7.3, 7.4, 7.0, 9.9, 10.9, 7.4, 9.9, 9.1,...",...,"[6, 3, 6, 3, 6, 6]","[6, 2, 6, 5, 6, 6]",2.0,1.0,2.0,1.0,3.0,3.0,2.0,I sometimes or often relied on the AI’s predic...


In [89]:
count = 0
to_remove = {}
for i, row in df.iterrows():
    a = row['user_decision_easy_mild']
    b = row['user_decision_easy_strong']
    c = row['user_decision_hard_mild']
    d = row['user_decision_hard_strong']
    if a == b or a == c or a == d or b == c or b == d or c == d:
        print(i)
        to_remove[i] = True
        count += 1

60535adfff835194160cd3bb
6662f201ccf41c36e7408562
5d59cb467ca41f0001e97e1f
5f2e72ed16a1ff1e83d98d1b
5f0219dce0d2de3a5179d1aa
5d0e091ccd9c9300191cb9c9
5b68c9eb87af310001584803
6458fe267afe71c7aa4de260
5e64e1d3ffed3225090e814d
6703aab642a723c1173593f3
5e459f61418f610891628564
6026e312c1e1234396ed6e7d
5bfd2d6b8acfed00011a5c89
5d68c8aa40524c00189e8ac2
63d13ca3a2e69ec8e19569f2
6658bac05e1a41dd4554301a
5ee5defbfe74a3000d106f51
651c1d7524852e9f9d614ce3
5eceb8411480c30fb89c7c54
56cf6e8d31a5bc0006e1cdf5
696b790c83b8fb2daf25674c
5fda057989ec4c35a700f69f
5f5bb87a0093333445af65f2
59b5a275c05bb50001eb3c4b
5e32c95c87b8ef28e5aff6ea
5c42f9388bb6fd000163d1b4
585586c728da4d0001884a00
5eea9efc609f0d1ad38199bd
63d3fc9de67e1179cdf0d080
634d7250a9dbe47b2b1d110f
599f00c54d25e800012fc9a6
5ef405ce2de83f1a2fa278e4
5ed0ff79d782b5000bda8eca
669053bbebafdd430e770142
5c641e21bcb1b40001863e9a
6730f965beb7513190d2be93
5fbf90f025a567296d943c7e
5d52e15363bdb100010883f1
63039343fd5a2e6fb88d2c61
6415cfa5324b6e802473285c


In [90]:
count

67

In [91]:
count = 0

for i, row in df.iterrows():
    a = row['answer_times_easy_mild']
    b = row['answer_times_easy_strong']
    c = row['answer_times_hard_mild']
    d = row['answer_times_hard_strong']

    if a == b or a == c or a == d or b == c or b == d or c == d:
        print(i)
        count += 1

In [92]:
count = 0

for i, row in df.iterrows():
    a = str(row['cogload_all_values_easy_mild'])
    b = str(row['cogload_all_values_easy_strong'])
    c = str(row['cogload_all_values_hard_mild'])
    d = str(row['cogload_all_values_hard_strong'])
    if a == b or a == c or a == d or b == c or b == d or c == d:
        print(i)
        to_remove[i] = True
        count += 1

59d3ef2c078dbe000195117f
5a59c1dfeedc32000142835a
6662f201ccf41c36e7408562
67607e34fb2abbf20ec24851
599a9252bbe848000179676e
55a13a6ffdf99b6ec83c23b8
5f639c21192b540a86e6e559
665b5d015bd1fbd411f8b77a
6659cb22b8e889fe11ae56d2
55ac3198fdf99b6de3b68b23
62b0a3fa60084743d22ad31c
5fbf94f2891c55203896f7dc
5ded58a8217786000b2b2511
5fb344483d8b23a8d2d106ca
5e459f61418f610891628564
5bfd2d6b8acfed00011a5c89
6663f3b2bc244a2d789b70a3
651c1d7524852e9f9d614ce3
59025280d17178000141a7dd
61505373bc903980a66ab1e1
5eceb8411480c30fb89c7c54
60dca45abca3eb229bafdedf
5a57e4f7acc75b00017a2dcd
67d00ceb0de1512e5669d4c5
62682277645054f5802459b8
60b6499af963de7ce527d399
5f6c423e8071b21b35b07ea6
64837ba51422d817b62cc5cc
5eab170759f5390b776df5a1
696b790c83b8fb2daf25674c
60819ba1a8862e33495f8914
671038fb26ff82aa6081d040
6361569d32bb406b8cf7de18
60aa79fa99f7d53701b02256
63874f631a1cdae0188d4e7a
665dc2d6a61b315970a3b030
599f00c54d25e800012fc9a6
670d2513894d1c633eeb11a6
5d960c43fbfe0500171450c7
67f8c966e81fca013fa2407a


In [93]:
count

63

In [94]:
len(to_remove.keys())

114

In [95]:
# List of keys to remove
keys_to_remove = to_remove.keys()

# Remove rows where "ID" is in keys_to_remove
df_filtered = df[~df.index.isin(keys_to_remove)]


In [96]:
df_filtered

,sex,age,xai_condition,tasks_order,comprehension_score,score_full,reliance_full,overreliance_full,underreliance_full,answer_times_full,...,cogload_all_values_hard_mild,cogload_all_values_hard_strong,trust_easy_mild,trust_easy_strong,trust_hard_mild,trust_hard_strong,quest_verify_answer_helpfulness,quest_understand_ai_processes_helpfulness,quest_ai_errors_detection_helpfulness,quest_ai_strategy
participant_id,,,,,,,,,,,,,,,,,,,,,
5a33e4a10758280001030ce5,Female,35,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",1.0,0.729167,0.645833,0.250,0.275,"[3.7, 6.2, 9.6, 5.8, 7.2, 12.7, 4.6, 4.2, 12.1...",...,"[4, 1, 4, 3, 5, 4]","[6, 2, 6, 5, 5, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5bcb95ffb592580001ec24cf,Female,43,H,"[hard_mild, easy_mild, easy_strong, hard_strong]",1.0,0.770833,0.666667,0.125,0.225,"[4.5, 8.2, 9.6, 7.7, 5.2, 14.5, 10.1, 6.4, 4.0...",...,"[6, 4, 6, 3, 6, 5]","[6, 6, 6, 5, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
631745a70c7c585248df1a5d,Male,33,H,"[hard_mild, hard_strong, easy_mild, easy_strong]",0.8,0.708333,0.625000,0.125,0.275,"[7.0, 10.0, 16.6, 20.6, 13.5, 10.9, 9.5, 10.5,...",...,"[3, 0, 6, 3, 5, 4]","[5, 0, 6, 3, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5fd8ea198512a801831f6b85,Male,40,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",1.0,0.770833,0.687500,0.250,0.225,"[6.7, 5.7, 7.4, 4.4, 5.6, 10.0, 7.6, 4.6, 9.8,...",...,"[5, 4, 3, 4, 4, 4]","[6, 6, 6, 6, 6, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
66d035e529e80315f590ecce,Female,46,H,"[hard_mild, easy_strong, hard_strong, easy_mild]",1.0,0.666667,0.625000,0.375,0.325,"[6.8, 7.8, 8.8, 7.0, 9.3, 9.1, 6.1, 6.0, 9.7, ...",...,"[5, 1, 4, 3, 5, 4]","[6, 1, 6, 4, 5, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5a9dd20b4eecca0001dd9f1d,Male,43,H+AI+GRADCAM,"[easy_strong, hard_mild, easy_mild, hard_strong]",1.0,0.833333,0.833333,0.500,0.100,"[5.5, 9.8, 14.2, 7.6, 9.0, 12.9, 11.0, 8.3, 4....",...,"[5, 2, 2, 4, 4, 4]","[5, 2, 6, 1, 5, 4]",3.0,2.0,3.0,3.0,2.0,2.0,4.0,I sometimes or often relied on the AI’s predic...
5dfb6c4d19232d899d182cda,Female,64,H+AI+GRADCAM,"[hard_strong, easy_strong, easy_mild, hard_mild]",0.8,0.770833,0.708333,0.250,0.200,"[6.9, 10.9, 17.8, 7.0, 7.3, 11.7, 5.2, 8.7, 6....",...,"[5, 2, 5, 5, 4, 5]","[4, 2, 5, 3, 1, 5]",3.0,3.0,3.0,3.0,4.0,3.0,5.0,I sometimes or often relied on the AI’s predic...
5fd89ebca76f0e0008c466a7,Female,32,H+AI+GRADCAM,"[hard_strong, easy_mild, hard_mild, easy_strong]",1.0,0.895833,0.770833,0.125,0.100,"[6.1, 5.6, 12.3, 5.3, 6.6, 5.0, 3.8, 5.5, 5.0,...",...,"[5, 0, 5, 2, 5, 4]","[6, 1, 6, 3, 6, 4]",2.0,1.0,2.0,2.0,3.0,4.0,5.0,I barely relied or did not rely at all on the ...
